In [1]:
import pandas as pd 

col_names = ["chr", "rsID", "GD", "pos", "ref", "alt"]
okg_snp_data = pd.read_csv("../../output/POP_STRATIFICATION/MERGE_DROP_VARIANTS/1kG_MDS8.bim", sep="\s+", names=col_names)
hapmap_snp_data = pd.read_csv("../../output/POP_STRATIFICATION/MERGE_DROP_VARIANTS/HapMap_MDS2.bim", sep="\s+", names=col_names)

In [20]:
parse_data = okg_snp_data.merge(hapmap_snp_data, how='outer', on="rsID", suffixes=["_okjp", "_hap"])
parse_data["check_match_chr"] = parse_data["chr_okjp"] == parse_data["chr_hap"]
parse_data["check_match_code"] = ((parse_data["ref_okjp"] == parse_data["ref_hap"]) & (parse_data["alt_okjp"] == parse_data["alt_hap"]))\
                                 | ((parse_data["ref_okjp"] == parse_data["alt_hap"]) & (parse_data["alt_okjp"] == parse_data["ref_hap"]))

# Check reverse complement
comp = {"A":"T", "T":"A", "C":"G", "G":"C"}
parse_data["check_revcomp"] = (
    (parse_data["ref_okjp"].map(comp) == parse_data["alt_hap"]) &
    (parse_data["alt_okjp"].map(comp) == parse_data["ref_hap"])
)

parse_data

,chr_okjp,rsID,GD_okjp,pos_okjp,ref_okjp,alt_okjp,chr_hap,GD_hap,pos_hap,ref_hap,alt_hap,check_match_chr,check_match_code,check_revcomp
0,12,rs1000000,0,125456933,A,G,12,0,125456933,C,T,True,False,True
1,4,rs10000010,0,21227772,C,T,4,0,21227772,A,G,True,False,True
2,4,rs10000012,0,1347325,G,C,4,0,1347325,G,C,True,True,True
3,3,rs1000002,0,185118462,T,C,3,0,185118462,G,A,True,False,True
4,4,rs10000023,0,95952929,T,G,4,0,95952929,C,A,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000988,4,rs9999953,0,41477753,A,G,4,0,41477753,C,T,True,False,True
1000989,4,rs9999966,0,5978256,T,C,4,0,5978256,G,A,True,False,True
1000990,4,rs9999979,0,135953341,T,C,4,0,135953341,G,A,True,False,True
1000991,4,rs9999992,0,123121534,A,G,4,0,123121534,C,T,True,False,True


In [21]:
# Check match chr
parse_data[~parse_data["check_match_chr"]]

,chr_okjp,rsID,GD_okjp,pos_okjp,ref_okjp,alt_okjp,chr_hap,GD_hap,pos_hap,ref_hap,alt_hap,check_match_chr,check_match_code,check_revcomp
319514,7,rs1640558,0,97808907,G,A,10,0,97808907,G,A,False,True,False


In [22]:
# Check allel code match
not_match_allele = parse_data[~parse_data["check_match_code"]]
not_match_allele

,chr_okjp,rsID,GD_okjp,pos_okjp,ref_okjp,alt_okjp,chr_hap,GD_hap,pos_hap,ref_hap,alt_hap,check_match_chr,check_match_code,check_revcomp
0,12,rs1000000,0,125456933,A,G,12,0,125456933,C,T,True,False,True
1,4,rs10000010,0,21227772,C,T,4,0,21227772,A,G,True,False,True
3,3,rs1000002,0,185118462,T,C,3,0,185118462,G,A,True,False,True
4,4,rs10000023,0,95952929,T,G,4,0,95952929,C,A,True,False,True
5,3,rs1000003,0,99825597,G,A,3,0,99825597,T,C,True,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1000988,4,rs9999953,0,41477753,A,G,4,0,41477753,C,T,True,False,True
1000989,4,rs9999966,0,5978256,T,C,4,0,5978256,G,A,True,False,True
1000990,4,rs9999979,0,135953341,T,C,4,0,135953341,G,A,True,False,True
1000991,4,rs9999992,0,123121534,A,G,4,0,123121534,C,T,True,False,True


In [5]:
parse_data[~parse_data["check_match_code"]]["check_revcomp"].count()

np.int64(922475)

Investigate Allele code changing

In [6]:
import pandas as pd

col_names = ["chr", "rsID", "GD", "pos", "ref", "alt"]

merging_okgp = pd.read_csv("../../output/POP_STRATIFICATION/MERGE_DROP_VARIANTS/1kG_MDS8.bim", sep="\s+", names=col_names).drop(columns="GD")
merging_hapmap = pd.read_csv("../../output/POP_STRATIFICATION/MERGE_DROP_VARIANTS/HapMap_MDS2.bim", sep="\s+", names=col_names).drop(columns="GD")

harmonized_okgp = pd.read_csv("../../output/POP_STRATIFICATION/HAMONIZE_OKGP_BUILD/1kG_MDS7.bim", sep="\s+", names=col_names).drop(columns="GD")
harmonized_hapmap = pd.read_csv("../../output/POP_STRATIFICATION/MERGE_ENSURE_HAPMAP_REF/HapMap-adj.bim", sep="\s+", names=col_names).drop(columns="GD")


hapmap_data = harmonized_hapmap.merge(merging_hapmap, how="inner", on="rsID", suffixes=["_harmonized_hapmap", "_merging_hapmap"])
hapmap_data = hapmap_data.merge(not_match_allele["rsID"], on="rsID", how="inner")

okgp_data = harmonized_okgp.merge(merging_okgp, how="inner", on="rsID", suffixes=["_harmonized_okgp", "_merging_okgp"])
okgp_data = okgp_data.merge(not_match_allele["rsID"], on="rsID", how="inner")

In [7]:
okgp_data

,chr_harmonized_okgp,rsID,pos_harmonized_okgp,ref_harmonized_okgp,alt_harmonized_okgp,chr_merging_okgp,pos_merging_okgp,ref_merging_okgp,alt_merging_okgp
0,1,rs3131969,744045,G,A,1,744045,G,A
1,1,rs1048488,750775,T,C,1,750775,T,C
2,1,rs12562034,758311,A,G,1,758311,A,G
3,1,rs12124819,766409,G,A,1,766409,G,A
4,1,rs4970383,828418,A,C,1,828418,A,C
...,...,...,...,...,...,...,...,...,...
922470,22,rs2285395,49524956,A,G,22,49524956,A,G
922471,22,rs13056621,49528625,G,A,22,49528625,G,A
922472,22,rs3888396,49558258,C,T,22,49558258,C,T
922473,22,rs2238837,49559741,C,A,22,49559741,C,A


In [8]:
# Check 1KGP changes
okgp_data["merging_okgp"] = okgp_data["ref_merging_okgp"] + okgp_data["alt_merging_okgp"]
okgp_data["harmonized_okgp"] = okgp_data["ref_harmonized_okgp"] + okgp_data["alt_harmonized_okgp"]
okgp_data.pivot_table(values="rsID", aggfunc="count", columns="merging_okgp", index="harmonized_okgp")

merging_okgp,AC,AG,AT,CA,CG,CT,GA,GC,GT,TA,TC,TG
harmonized_okgp,,,,,,,,,,,,
AC,43820.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AG,NaN,192242.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AT,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CA,NaN,NaN,NaN,42496.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CG,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CT,NaN,NaN,NaN,NaN,NaN,183435.0,NaN,NaN,NaN,NaN,NaN,NaN
GA,NaN,NaN,NaN,NaN,NaN,NaN,182202.0,NaN,NaN,NaN,NaN,NaN
GC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN
GT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42358.0,NaN,NaN,NaN


In [9]:
# Check Hapmap changes
hapmap_data["merging_hapmap"] = hapmap_data["ref_merging_hapmap"] + hapmap_data["alt_merging_hapmap"]
hapmap_data["harmonized_hapmap"] = hapmap_data["ref_harmonized_hapmap"] + hapmap_data["alt_harmonized_hapmap"]
hapmap_data.pivot_table(values="rsID", aggfunc="count", columns="merging_hapmap", index="harmonized_hapmap")

merging_hapmap,AC,AG,CA,CT,GA,GC,GT,TC,TG
harmonized_hapmap,,,,,,,,,
AC,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,42499.0
AG,NaN,NaN,NaN,1.0,NaN,NaN,NaN,182207.0,NaN
CA,NaN,NaN,NaN,NaN,NaN,NaN,43821.0,NaN,NaN
CG,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN
CT,NaN,2.0,NaN,NaN,192312.0,NaN,NaN,NaN,NaN
GA,NaN,NaN,NaN,192241.0,NaN,NaN,NaN,2.0,NaN
GT,NaN,NaN,43593.0,NaN,NaN,NaN,NaN,NaN,NaN
TC,NaN,183431.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN
TG,42360.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
## Check convertable of hapmap and 1KGP after being harmonized
hapmap_data[["rsID", "harmonized_hapmap"]]\
        .merge(okgp_data[["rsID", "harmonized_okgp"]], on="rsID", how="inner")\
        .pivot_table(values="rsID", aggfunc="count", columns="harmonized_hapmap", index="harmonized_okgp")

harmonized_hapmap,AC,AG,CA,CG,CT,GA,GT,TC,TG
harmonized_okgp,,,,,,,,,
AC,NaN,NaN,43818.0,NaN,NaN,2.0,NaN,NaN,NaN
AG,NaN,NaN,NaN,1.0,NaN,192237.0,NaN,NaN,4.0
AT,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN
CA,42493.0,NaN,NaN,NaN,NaN,2.0,NaN,1.0,NaN
CG,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CT,1.0,NaN,NaN,NaN,NaN,NaN,3.0,183430.0,1.0
GA,NaN,182199.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN
GC,1.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
GT,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,42356.0


In [11]:
## Check convertable of hapmap and 1KGP ready for merging
hapmap_data[["rsID", "merging_hapmap"]]\
        .merge(okgp_data[["rsID", "merging_okgp"]], on="rsID", how="inner")\
        .pivot_table(values="rsID", aggfunc="count", columns="merging_hapmap", index="merging_okgp")

merging_hapmap,AC,AG,CA,CT,GA,GC,GT,TC,TG
merging_okgp,,,,,,,,,
AC,NaN,NaN,NaN,2.0,NaN,NaN,43818.0,NaN,NaN
AG,4.0,NaN,NaN,192237.0,NaN,1.0,NaN,NaN,NaN
AT,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
CA,NaN,1.0,NaN,2.0,NaN,NaN,NaN,NaN,42493.0
CG,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,1.0
CT,NaN,183430.0,4.0,NaN,NaN,NaN,NaN,NaN,1.0
GA,NaN,NaN,NaN,NaN,NaN,NaN,3.0,182199.0,NaN
GC,NaN,NaN,NaN,NaN,2.0,NaN,1.0,NaN,NaN
GT,42356.0,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN


In [49]:
## Create function to test multiple cases

def check_trans(bim1, bim2, name1, name2, use_ref_snps=True):
    col_names = ["chr", "rsID", "GD", "pos", "ref", "alt"]

    bim1_df = pd.read_csv(bim1, sep="\s+", names=col_names)
    bim2_df = pd.read_csv(bim2, sep="\s+", names=col_names)
    merge_table = bim1_df.merge(bim2_df, how="inner", on="rsID", suffixes=[f"_{name1}", f"_{name2}"])
    if use_ref_snps:
        merge_table = merge_table.merge(not_match_allele["rsID"], on="rsID", how="inner")  # filter unmatch SNPs 

    merge_table[f"{name1}_concat"] = (merge_table[f"ref_{name1}"] + merge_table[f"alt_{name1}"]).map(lambda x: "".join(sorted(x)))
    merge_table[f"{name2}_concat"] = (merge_table[f"ref_{name2}"] + merge_table[f"alt_{name2}"]).map(lambda x: "".join(sorted(x)))
    pivot_df = merge_table.pivot_table(values="rsID", aggfunc="count", columns=f"{name1}_concat", index=f"{name2}_concat")

    return pivot_df

check_trans(
    "../../output/POP_STRATIFICATION/MERGE_DROP_VARIANTS/1kG_MDS8.bim",
    "../../output/POP_STRATIFICATION/MERGE_DROP_VARIANTS/HapMap_MDS2.bim",
    "merging_1kgp",
    "merging_hapmap"
)

merging_1kgp_concat,AC,AG,AT,CG,CT,GT
merging_hapmap_concat,,,,,,
AC,NaN,4.0,NaN,NaN,4.0,85946.0
AG,1.0,NaN,2.0,2.0,375740.0,2.0
CG,NaN,1.0,NaN,NaN,NaN,1.0
CT,4.0,374436.0,1.0,1.0,NaN,9.0
GT,86311.0,3.0,NaN,2.0,5.0,NaN


In [50]:
HapMap_QC = "../../output/QC_GWAS/RELATEDNESS_DELETE_RELATED_SAMPLES/HapMap_3_r3_12.bim"
HapMap_Hamonized = "../../output/POP_STRATIFICATION/HAMONIZE_HAPMAP_VARIANT/HapMap_MDS.bim"
HapMap_Merging = "../../output/POP_STRATIFICATION/MERGE_DROP_VARIANTS/HapMap_MDS2.bim"

OKGP_QC = "../../output/POP_STRATIFICATION/OKGP_QC_MAF/1kG_MDS5.bim" 
OKGP_Hamonized = "../../output/POP_STRATIFICATION/HAMONIZE_OKGP_BUILD/1kG_MDS7.bim"
OKGP_Merging = "../../output/POP_STRATIFICATION/MERGE_DROP_VARIANTS/1kG_MDS8.bim"

In [51]:
## Check HapMap_QC vs OKGP_QC
check_trans(HapMap_QC, OKGP_QC, "HapMap_QC", "OKGP_QC")

HapMap_QC_concat,AC,AG,CG,CT,GT
OKGP_QC_concat,,,,,
AC,86311.0,4.0,NaN,1.0,NaN
AG,3.0,374436.0,1.0,NaN,4.0
AT,NaN,1.0,NaN,2.0,NaN
CG,2.0,1.0,NaN,2.0,NaN
CT,5.0,NaN,NaN,375740.0,4.0
GT,NaN,9.0,1.0,2.0,85946.0


In [52]:
## Check HapMap_Hamonized vs OKGP_Hamonized
check_trans(HapMap_Hamonized, OKGP_Hamonized, "HapMap_Hamonized", "OKGP_Hamonized")

HapMap_Hamonized_concat,AC,AG,CG,CT,GT
OKGP_Hamonized_concat,,,,,
AC,86311.0,4.0,NaN,1.0,NaN
AG,3.0,374436.0,1.0,NaN,4.0
AT,NaN,1.0,NaN,2.0,NaN
CG,2.0,1.0,NaN,2.0,NaN
CT,5.0,NaN,NaN,375740.0,4.0
GT,NaN,9.0,1.0,2.0,85946.0


In [53]:
## Check HapMap_Merging vs OKGP_Merging
check_trans(HapMap_Merging, OKGP_Merging, "HapMap_Merging", "OKGP_Merging")

HapMap_Merging_concat,AC,AG,CG,CT,GT
OKGP_Merging_concat,,,,,
AC,NaN,1.0,NaN,4.0,86311.0
AG,4.0,NaN,1.0,374436.0,3.0
AT,NaN,2.0,NaN,1.0,NaN
CG,NaN,2.0,NaN,1.0,2.0
CT,4.0,375740.0,NaN,NaN,5.0
GT,85946.0,2.0,1.0,9.0,NaN


More intensive investigate on 2 processes:
- MERGE_ENSURE_HAPMAP_REF
- MERGE_STRAND_PB_FLIP

In [54]:
# MERGE_ENSURE_HAPMAP_REF
hapmap_file = "../../output/POP_STRATIFICATION/MERGE_ENSURE_HAPMAP_REF/HapMap-adj.bim"
okgp_file = "../../output/POP_STRATIFICATION/HAMONIZE_OKGP_BUILD/1kG_MDS7.bim"

check_trans(hapmap_file, okgp_file, "HapMap_adj", "1kG_MDS7", use_ref_snps=False)

HapMap_adj_concat,AC,AG,AT,CG,CT,GT
1kG_MDS7_concat,,,,,,
AC,86311.0,4.0,NaN,NaN,1.0,85.0
AG,3.0,374436.0,NaN,1.0,304.0,4.0
AT,NaN,1.0,31630.0,NaN,2.0,NaN
CG,2.0,1.0,NaN,46118.0,2.0,NaN
CT,5.0,308.0,NaN,NaN,375740.0,4.0
GT,73.0,9.0,NaN,1.0,2.0,85946.0


In [55]:
# MERGE_STRAND_PB_CHECK + MERGE_STRAND_PB_FLIP
hapmap_file="../../output/POP_STRATIFICATION/MERGE_STRAND_PB_FLIP/corrected_hapmap.bim"
okgp_file = "../../output/POP_STRATIFICATION/HAMONIZE_OKGP_BUILD/1kG_MDS7.bim"

check_trans(hapmap_file, okgp_file, "HapMap_corrected", "1kG_MDS7", use_ref_snps=False)

HapMap_corrected_concat,AC,AG,AT,CG,CT,GT
1kG_MDS7_concat,,,,,,
AC,85.0,1.0,NaN,NaN,4.0,86311.0
AG,4.0,304.0,NaN,1.0,374436.0,3.0
AT,NaN,2.0,31630.0,NaN,1.0,NaN
CG,NaN,2.0,NaN,46118.0,1.0,2.0
CT,4.0,375740.0,NaN,NaN,308.0,5.0
GT,85946.0,2.0,NaN,1.0,9.0,73.0
